In [134]:
import pandas as pd
import numpy as np

In [135]:
df1 = pd.read_excel('test/model1.xlsx', header=None)
df2 = pd.read_excel('test/model2.xlsx', header=None)

In [136]:
df1.iloc[0, 0] = 'model'
df2.iloc[0, 0] = 'model'

In [137]:
df1.columns = df1.iloc[0]
df2.columns = df2.iloc[0]
df1 = df1[1:]
df2 = df2[1:] 

In [138]:
df1

,model,Alicia 1013,Alicia 1013
1,AAA,1,7
2,BBB,2,8
3,CCC,3,9
4,DDD,4,10
5,EEE,5,11
6,FFF,6,12


In [139]:
df2

,model,Alicia 1013A,Alicia 1013A
1,AAA,1,7
2,NaN,2,8
3,BBB,3,9
4,NaN,4,10
5,GGG,5,11
6,HHH,6,12


In [140]:
df2['model_filled'] = df2['model'].fillna(method='ffill')

# 番号付きサフィックスを追加
counter = {}
result = []
for original, filled in zip(df2['model'], df2['model_filled']):
    if pd.isna(original):  # 元が欠損値であればサフィックス追加
        counter[filled] = counter.get(filled, 0) + 1
        result.append(f"{filled}{counter[filled]}")
    else:
        result.append(filled)

df2['model_with_suffix'] = result

# model 列を更新し、作成した列を削除
df2['model'] = df2['model_with_suffix']
df2.drop(columns=['model_filled', 'model_with_suffix'], inplace=True)

/var/folders/gx/lc33vdvs5dj0sf9x1njqzryr0000gn/T/ipykernel_25576/3016495456.py:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df2['model_filled'] = df2['model'].fillna(method='ffill')


In [141]:
df1['model2'] = df1.model
df2['model2'] = df2.model

In [142]:
# df1 = df1.reset_index()
# df2 = df2.reset_index()

In [143]:
df1 = df1.set_index(['model','model2'])
df2 = df2.set_index(['model','model2'])

In [144]:
df_merge = pd.concat([df2,df1],axis=1)

In [145]:
df_merge

,,Alicia 1013A,Alicia 1013A,Alicia 1013,Alicia 1013
model,model2,,,,
AAA,AAA,1,7,1,7
AAA1,AAA1,2,8,NaN,NaN
BBB,BBB,3,9,2,8
BBB1,BBB1,4,10,NaN,NaN
GGG,GGG,5,11,NaN,NaN
HHH,HHH,6,12,NaN,NaN
CCC,CCC,NaN,NaN,3,9
DDD,DDD,NaN,NaN,4,10
EEE,EEE,NaN,NaN,5,11


In [146]:
df_merge = df_merge.T

In [147]:
df_merge

model,AAA,AAA1,BBB,BBB1,GGG,HHH,CCC,DDD,EEE,FFF
model2,AAA,AAA1,BBB,BBB1,GGG,HHH,CCC,DDD,EEE,FFF
0,,,,,,,,,,
Alicia 1013A,1,2,3,4,5,6,NaN,NaN,NaN,NaN
Alicia 1013A,7,8,9,10,11,12,NaN,NaN,NaN,NaN
Alicia 1013,1,NaN,2,NaN,NaN,NaN,3,4,5,6
Alicia 1013,7,NaN,8,NaN,NaN,NaN,9,10,11,12


In [121]:
df_merge.reset_index(drop=True, inplace=True)

In [61]:
df_merge.index.name = 'A'

In [88]:
df_merge.to_csv("出力ファイル名.csv",  encoding="utf-8")

In [148]:
# 結合する
import pandas as pd
import os

# フォルダ内のCSVファイルを処理する関数
def process_csv_files(folder_path):
    df_list = []  # データフレームを保存するリスト

    # フォルダ内のすべてのCSVファイルを取得
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.xlsx'):
            file_path = os.path.join(folder_path, file_name)
            
            # CSVを読み込んで処理
            df = pd.read_excel(file_path, header=None ,index_col=0)
            
            df.index = df.index.fillna("model")
            df_list.append(df)
    
    # 全てのデータフレームを連結
    df_merge = pd.concat(df_list,axis=1)
    return df_merge

# フォルダのパスを指定して関数を実行
folder_path = 'test'  # フォルダのパスを入力
result_df = process_csv_files(folder_path)
result_df = result_df.T

# 結果を確認
result_df.to_csv("出力ファイル名.csv", index=False, encoding="utf-8")


InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [237]:
# 穴埋めして結合する
import pandas as pd
import os

# フォルダ内のCSVファイルを処理する関数
def process_csv_files(folder_path):
    df_list = []  # データフレームを保存するリスト

    # フォルダ内のすべてのCSVファイルを取得
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.xlsx'):
            file_path = os.path.join(folder_path, file_name)
            
            # CSVを読み込んで処理
            df = pd.read_excel(file_path, header=None)
            df.iloc[0, 0] = 'model'
            df.columns = df.iloc[0]
            df = df[1:] 
            df['model_filled'] = df['model'].ffill()

            # 番号付きサフィックスを追加
            counter = {}
            result = []
            for original, filled in zip(df['model'], df['model_filled']):
                if pd.isna(original):  # 元が欠損値であればサフィックス追加
                    counter[filled] = counter.get(filled, 0) + 1
                    result.append(f"{filled}{counter[filled]}")
                else:
                    result.append(filled)

            df['model_with_suffix'] = result

            # model 列を更新し、作成した列を削除
            df['model'] = df['model_with_suffix']
            df.drop(columns=['model_filled', 'model_with_suffix'], inplace=True)
            df['model2'] = df.model
            # df = df.set_index(['model','model2'])
            df.set_index(["model", "model2"], inplace=True)
            df_list.append(df)
    
    # 全てのデータフレームを連結
    df_merge = pd.concat(df_list,axis=1)
    return df_merge

# フォルダのパスを指定して関数を実行
folder_path = 'test'  # フォルダのパスを入力
result_df = process_csv_files(folder_path)
result_df = result_df.reset_index()
result_df = result_df.set_index(['model2'])
result_df = result_df.T
result_df.reset_index()

# 結果を確認
result_df.to_csv("出力ファイル名.csv", encoding="utf-8")

In [238]:
df=pd.read_csv("出力ファイル名.csv", encoding="utf-8",header=1)
df

,model,AAA,AAA1,BBB,BBB1,GGG,HHH,CCC,DDD,EEE,FFF
0,Alicia 1013A,1,2.0,3,4.0,5.0,6.0,NaN,NaN,NaN,NaN
1,Alicia 1013A,7,8.0,9,10.0,11.0,12.0,NaN,NaN,NaN,NaN
2,Alicia 1013,1,NaN,2,NaN,NaN,NaN,3.0,4.0,5.0,6.0
3,Alicia 1013,7,NaN,8,NaN,NaN,NaN,9.0,10.0,11.0,12.0


In [241]:
result_df

model,AAA,AAA1,BBB,BBB1,GGG,HHH,CCC,DDD,EEE,FFF
0,,,,,,,,,,
model,AAA,AAA1,BBB,BBB1,GGG,HHH,CCC,DDD,EEE,FFF
Alicia 1013A,1,2,3,4,5,6,NaN,NaN,NaN,NaN
Alicia 1013A,7,8,9,10,11,12,NaN,NaN,NaN,NaN
Alicia 1013,1,NaN,2,NaN,NaN,NaN,3,4,5,6
Alicia 1013,7,NaN,8,NaN,NaN,NaN,9,10,11,12


In [209]:
stacked_table = result_df.set_index(["model", "model2"]).stack().reset_index(name="A")
stacked_table = stacked_table.rename(columns={0:'mo'})
stacked_table = stacked_table.set_index(['mo'])
stacked_table

,model,model2,A
mo,,,
Alicia 1013A,AAA,AAA,1
Alicia 1013A,AAA,AAA,7
Alicia 1013,AAA,AAA,1
Alicia 1013,AAA,AAA,7
Alicia 1013A,AAA1,AAA1,2
Alicia 1013A,AAA1,AAA1,8
Alicia 1013A,BBB,BBB,3
Alicia 1013A,BBB,BBB,9
Alicia 1013,BBB,BBB,2


In [167]:
result_df.index

Index(['Alicia 1013A', 'Alicia 1013A', 'Alicia 1013', 'Alicia 1013'], dtype='object', name=0)

In [ ]:
import win32com.client
import os

# Excelファイルを開く
excel = win32com.client.Dispatch("Excel.Application")
workbook = excel.Workbooks.Open('ファイルパス.xls')
sheet = workbook.Sheets(1)

# 保存フォルダの指定
output_folder = "C:/Path/To/Save/Images"
os.makedirs(output_folder, exist_ok=True)  # フォルダがない場合は作成

# 保存済みモデル名称を記録するセット
saved_models = set()

# モデル名称と画像を保存する処理
for shape in sheet.Shapes:
    if shape.Type == 13:  # Picture
        # 画像のセル位置を取得
        cell_address = shape.TopLeftCell.Address
        model_name = sheet.Range(cell_address).Value  # モデル名称の取得
        
        if model_name and model_name not in saved_models:
            # モデル名称をセットに追加
            saved_models.add(model_name)
            
            # ファイル名を作成して画像を保存
            file_name = f"{model_name}.png"
            file_path = os.path.join(output_folder, file_name)
            
            shape.Copy()
            with open(file_path, "wb") as f:
                f.write(shape.PictureFormat.Picture())
            print(f"画像を保存しました: {file_path}")
        elif model_name:
            print(f"モデル名称 '{model_name}' の画像は既に保存されています。")
        else:
            print(f"セル {cell_address} にモデル名称が見つかりません。")
